In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark import StorageLevel

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [3]:
# !pip install scikit-plot

In [4]:
import sklearn
import scikitplot as skplt
from sklearn.metrics import classification_report, confusion_matrix, precision_score

<hr />
<hr />
<hr />

In [5]:
result_schema = StructType([
                    StructField('experiment_filter', StringType(), True),
                    StructField('undersampling_method', StringType(), True),
                    StructField('undersampling_column', StringType(), True),
                    StructField('filename', StringType(), True),
                    StructField('experiment_id', StringType(), True),
                    StructField('n_covid', IntegerType(), True),
                    StructField('n_not_covid', IntegerType(), True),
                    StructField('model_name', StringType(), True),
                    StructField('model_seed', StringType(), True),
                    StructField('model_maxIter', IntegerType(), True),
                    StructField('model_maxDepth', IntegerType(), True),
                    StructField('model_maxBins', IntegerType(), True),
                    StructField('model_minInstancesPerNode', IntegerType(), True),
                    StructField('model_minInfoGain', FloatType(), True),
                    StructField('model_featureSubsetStrategy', StringType(), True),
                    StructField('model_n_estimators', IntegerType(), True),
                    StructField('model_learning_rate', FloatType(), True),
                    StructField('model_impurity', StringType(), True),
                    StructField('model_AUC_ROC', StringType(), True),
                    StructField('model_AUC_PR', StringType(), True),
                    StructField('model_covid_precision', StringType(), True),
                    StructField('model_covid_recall', StringType(), True),
                    StructField('model_covid_f1', StringType(), True),
                    StructField('model_not_covid_precision', StringType(), True),
                    StructField('model_not_covid_recall', StringType(), True),
                    StructField('model_not_covid_f1', StringType(), True),
                    StructField('model_avg_precision', StringType(), True),
                    StructField('model_avg_recall', StringType(), True),
                    StructField('model_avg_f1', StringType(), True),
                    StructField('model_avg_acc', StringType(), True),
                    StructField('model_TP', StringType(), True),
                    StructField('model_TN', StringType(), True),
                    StructField('model_FN', StringType(), True),
                    StructField('model_FP', StringType(), True),
                    StructField('model_time_exec', StringType(), True),
                    StructField('model_col_set', StringType(), True)
                          ])

<hr />
<hr />
<hr />

In [6]:
cols_sets = ['cols_set_1', 'cols_set_2', 'cols_set_3']
undersamp_col = ['03-STRSAMP-AG', '04-STRSAMP-EW']
dfs = ['ds-2'] #, 'ds-2', 'ds-3']

In [7]:
# lists of params
model_numTrees = [20, 50] 
model_maxDepth = [3, 5, 7] 
model_maxBins = [32, 64]

list_of_param_dicts = []

for numTrees in model_numTrees:
    for maxDepth in model_maxDepth:
        for maxBins in model_maxBins: 
            params_dict = {}
            params_dict['numTrees'] = numTrees
            params_dict['maxDepth'] = maxDepth
            params_dict['maxBins'] = maxBins
            list_of_param_dicts.append(params_dict)

print("There is {} set of params.".format(len(list_of_param_dicts)))
# list_of_param_dicts

There is 12 set of params.


In [8]:
prefix = 'gs://ai-covid19-datalake/trusted/experiment_map/'

<hr />
<hr />
<hr />

In [9]:
# filename = 'gs://ai-covid19-datalake/trusted/experiment_map/03-STRSAMP-AG/ds-1/cols_set_1/experiment0.parquet'
# df = spark.read.parquet(filename)
# df.limit(2).toPandas()

In [10]:
# params_dict = {'numTrees': 50,
#                'maxDepth': 3,
#                'maxBins': 32}
# cols = 'cols_set_1'
# experiment_filter = 'ds-1'
# undersampling_method = '03-STRSAMP-AG', 
# experiment_id = 0

In [11]:
# run_rf(df, params_dict, cols, filename, experiment_filter, undersampling_method, experiment_id)

<hr />
<hr />
<hr />

In [12]:
def run_rf(exp_df, params_dict, cols, filename, experiment_filter, 
            undersampling_method, experiment_id):
    import time
    start_time = time.time()
    
    n_covid = exp_df.filter(F.col('CLASSI_FIN') == 1.0).count()
    n_not_covid = exp_df.filter(F.col('CLASSI_FIN') == 0.0).count()
    
    
    id_cols = ['NU_NOTIFIC', 'CLASSI_FIN']

    labelIndexer = StringIndexer(inputCol="CLASSI_FIN", outputCol="indexedLabel").fit(exp_df)    
    
    input_cols = [x for x in exp_df.columns if x not in id_cols]
    assembler = VectorAssembler(inputCols = input_cols, outputCol= 'features')
    exp_df = assembler.transform(exp_df)
    
    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=30).fit(exp_df)
    
    # Split the data into training and test sets (30% held out for testing)
    (trainingData, testData) = exp_df.randomSplit([0.7, 0.3])
    trainingData = trainingData.persist(StorageLevel.MEMORY_ONLY)
    testData = testData.persist(StorageLevel.MEMORY_ONLY)
    
    # Train a RandomForest model.
    rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                               numTrees = params_dict['numTrees'], 
                               maxDepth = params_dict['maxDepth'], 
                               maxBins = params_dict['maxBins'])
    
    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

    # Chain indexers and forest in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

    # Train model.  This also runs the indexers.
    model = pipeline.fit(trainingData)
    
    # Make predictions.
    predictions = model.transform(testData)    
    
    
    pred = predictions.select(['CLASSI_FIN', 'predictedLabel'])\
                  .withColumn('predictedLabel', F.col('predictedLabel').cast('double'))\
                  .withColumn('predictedLabel', F.when(F.col('predictedLabel') == 1.0, 'covid').otherwise('n-covid'))\
                  .withColumn('CLASSI_FIN', F.when(F.col('CLASSI_FIN') == 1.0, 'covid').otherwise('n-covid'))\
                  .toPandas()

    y_true = pred['CLASSI_FIN'].tolist()
    y_pred = pred['predictedLabel'].tolist()
    
    report = classification_report(y_true, y_pred, output_dict=True)
    
    
    evaluator_ROC = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
    accuracy_ROC = evaluator_ROC.evaluate(predictions)


    
    evaluator_PR = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderPR")
    accuracy_PR = evaluator_PR.evaluate(predictions)
    
    conf_matrix = confusion_matrix(y_true, y_pred)

    result_dict = {}
    
    result_dict['experiment_filter'] = experiment_filter
    result_dict['undersampling_method'] = undersampling_method
    result_dict['filename'] = filename
    result_dict['experiment_id'] = experiment_id
    result_dict['n_covid'] = n_covid
    result_dict['n_not_covid'] = n_not_covid
    result_dict['model_name'] = 'RF'
    result_dict['params'] = params_dict
    result_dict['model_AUC_ROC'] = accuracy_ROC
    result_dict['model_AUC_PR'] = accuracy_PR
    result_dict['model_covid_precision'] = report['covid']['precision']
    result_dict['model_covid_recall'] = report['covid']['recall']
    result_dict['model_covid_f1'] = report['covid']['f1-score']
    result_dict['model_not_covid_precision'] = report['n-covid']['precision']
    result_dict['model_not_covid_recall'] = report['n-covid']['recall']
    result_dict['model_not_covid_f1'] = report['n-covid']['f1-score']
    result_dict['model_avg_precision'] = report['macro avg']['precision']
    result_dict['model_avg_recall'] = report['macro avg']['recall']
    result_dict['model_avg_f1'] = report['macro avg']['f1-score']
    result_dict['model_avg_acc'] = report['accuracy']
    result_dict['model_TP'] = conf_matrix[0][0]
    result_dict['model_TN'] = conf_matrix[1][1]
    result_dict['model_FN'] = conf_matrix[0][1]
    result_dict['model_FP'] = conf_matrix[1][0]
    result_dict['model_time_exec'] = time.time() - start_time
    result_dict['model_col_set'] = cols
    
    return result_dict

<hr />
<hr />
<hr />

# Running GBT on 10 samples for each experiment
### 3x col sets -> ['cols_set_1', 'cols_set_2', 'cols_set_3']
### 3x model_maxIter -> [100, 200, 300]
### 3x model_maxDepth -> [5, 10, 15]
### 3x model_maxBins -> [16, 32, 64]
Total: 10 * 3 * 3 * 3 * 3 = 810

In [13]:
experiments = []

### Datasets: strat_samp_lab_agegrp

In [ ]:
for uc in undersamp_col: 
    for ds in dfs:
        for col_set in cols_sets:
            for params_dict in list_of_param_dicts: 
                for id_exp in range(50):
                    filename = prefix + uc + '/' + ds + '/' + col_set + '/' + 'experiment' + str(id_exp) + '.parquet'
                    exp_dataframe = spark.read.parquet(filename)
                    print('read {}'.format(filename))
                    
                    undersampling_method = uc
                    experiment_filter = ds
                    experiment_id = id_exp

                    try:                     
                        model = run_rf(exp_dataframe, params_dict, col_set, filename, experiment_filter, undersampling_method, experiment_id)
                        experiments.append(model)

                        print("Parameters ==> {}\n Results: \n AUC_PR: {} \n Precision: {} \n Time: {}".format(str(params_dict), str(model['model_AUC_PR']), str(model['model_avg_precision']), str(model['model_time_exec'])))
                        print('=========================== \n')
                    except:
                        print('=========== W A R N I N G =========== \n')
                        print('Something wrong with the exp: {}, {}, {}'.format(filename, params_dict, col_set))

read gs://ai-covid19-datalake/trusted/experiment_map/03-STRSAMP-AG/ds-2/cols_set_1/experiment0.parquet
Parameters ==> {'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.91654728051533 
 Precision: 0.9150798291076228 
 Time: 20.483246088027954

read gs://ai-covid19-datalake/trusted/experiment_map/03-STRSAMP-AG/ds-2/cols_set_1/experiment1.parquet
Parameters ==> {'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.916152758117841 
 Precision: 0.9160715753172187 
 Time: 9.032374382019043

read gs://ai-covid19-datalake/trusted/experiment_map/03-STRSAMP-AG/ds-2/cols_set_1/experiment2.parquet
Parameters ==> {'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.9195147037189743 
 Precision: 0.9190491750360557 
 Time: 8.734893083572388

read gs://ai-covid19-datalake/trusted/experiment_map/03-STRSAMP-AG/ds-2/cols_set_1/experiment3.parquet
Parameters ==> {'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}
 Results: 
 AUC_PR: 0.9189439628150644 
 Precisi

<hr />
<hr />
<hr />

In [ ]:
for i in range(len(experiments)):
    for d in list(experiments[i].keys()):
        experiments[i][d] = str(experiments[i][d])

In [ ]:
# experiments

In [ ]:
cols = ['experiment_filter', 'undersampling_method', 'filename', 'experiment_id', 'n_covid', 'n_not_covid', 'model_name', 'params', 'model_AUC_ROC', 'model_AUC_PR', 'model_covid_precision', 'model_covid_recall', 'model_covid_f1', 'model_not_covid_precision', 'model_not_covid_recall', 'model_not_covid_f1', 'model_avg_precision', 'model_avg_recall', 'model_avg_f1', 'model_avg_acc', 'model_TP', 'model_TN', 'model_FN', 'model_FP', 'model_time_exec', 'model_col_set']

In [ ]:
intermed_results = spark.createDataFrame(data=experiments).select(cols)
intermed_results.toPandas()

/usr/lib/spark/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


,experiment_filter,undersampling_method,filename,experiment_id,n_covid,n_not_covid,model_name,params,model_AUC_ROC,model_AUC_PR,model_covid_precision,model_covid_recall,model_covid_f1,model_not_covid_precision,model_not_covid_recall,model_not_covid_f1,model_avg_precision,model_avg_recall,model_avg_f1,model_avg_acc,model_TP,model_TN,model_FN,model_FP,model_time_exec,model_col_set
0,ds-2,03-STRSAMP-AG,gs://ai-covid19-datalake/trusted/experiment_ma...,0,53203,44538,RF,"{'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}",0.9003703148796962,0.91654728051533,0.8713406993962866,0.9697641987829615,0.9179216415671687,0.9588189588189588,0.8309764309764309,0.8903318903318903,0.9150798291076228,0.9003703148796962,0.9041267659495296,0.9061116639785869,15299,11106,477,2259,20.483246088027954,cols_set_1
1,ds-2,03-STRSAMP-AG,gs://ai-covid19-datalake/trusted/experiment_ma...,1,53038,44538,RF,"{'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}",0.9005234599047575,0.916152758117841,0.8732497328909633,0.9703805536461914,0.9192564967738117,0.9588934177434741,0.8306663661633236,0.8901859753643023,0.9160715753172187,0.9005234599047575,0.9047212360690571,0.9069386641195333,15529,11057,474,2254,9.032374382019043,cols_set_1
2,ds-2,03-STRSAMP-AG,gs://ai-covid19-datalake/trusted/experiment_ma...,2,53459,44538,RF,"{'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}",0.9027933798239248,0.9195147037189743,0.8758045558851514,0.9731343283582089,0.9219076796182284,0.9622937941869599,0.8324524312896406,0.8926764098619491,0.9190491750360557,0.9027933798239247,0.9072920447400887,0.9095962351657345,15648,11025,432,2219,8.734893083572388,cols_set_1
3,ds-2,03-STRSAMP-AG,gs://ai-covid19-datalake/trusted/experiment_ma...,3,52952,44538,RF,"{'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}",0.901473278399856,0.9189439628150644,0.8728635511895975,0.9725420726306466,0.9200107729598708,0.9619198034570501,0.8304044841690653,0.8913370462213911,0.9173916773233237,0.901473278399856,0.905673909590631,0.9078530060672918,15372,10963,434,2239,8.61351227760315,cols_set_1
4,ds-2,03-STRSAMP-AG,gs://ai-covid19-datalake/trusted/experiment_ma...,4,52087,44538,RF,"{'numTrees': 20, 'maxDepth': 3, 'maxBins': 32}",0.9015033494283551,0.9228861170152995,0.8677309665148982,0.9748138556167044,0.9181607513111355,0.9660321341250436,0.828192843240006,0.8918178153970173,0.916881550319971,0.9015033494283552,0.9049892833540765,0.9068152622990661,15056,11063,389,2295,8.599029779434204,cols_set_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,ds-2,04-STRSAMP-EW,gs://ai-covid19-datalake/trusted/experiment_ma...,45,52202,44538,RF,"{'numTrees': 50, 'maxDepth': 7, 'maxBins': 64}",0.9105240049772938,0.9187945054643145,0.887750349324639,0.9658579844175588,0.9251585110578527,0.9548008385744234,0.8551900255370287,0.9022544474820714,0.9212755939495312,0.9105240049772938,0.9137064792699621,0.9152262808838184,15248,11386,539,1928,13.470989465713501,cols_set_3
3596,ds-2,04-STRSAMP-EW,gs://ai-covid19-datalake/trusted/experiment_ma...,46,52640,44538,RF,"{'numTrees': 50, 'maxDepth': 7, 'maxBins': 64}",0.9117607604791406,0.9203868775448641,0.8885767790262172,0.9665181413112667,0.9259101164705166,0.9559353271341208,0.8570033796470147,0.9037699984159671,0.922256053080169,0.9117607604791407,0.9148400574432418,0.9162790697674419,15184,11411,526,1904,12.356247186660767,cols_set_3
3597,ds-2,04-STRSAMP-EW,gs://ai-covid19-datalake/trusted/experiment_ma...,47,51712,44538,RF,"{'numTrees': 50, 'maxDepth': 7, 'maxBins': 64}",0.9080123203919233,0.9175793429014565,0.8830642794247138,0.9650372081088017,0.9222327805535971,0.9542823588625116,0.8509874326750448,0.8996797026375104,0.9186733191436127,0.9080123203919233,0.9109562415955538,0.9123843072247548,15043,11376,545,1992,19.11794924736023,cols_set_3
3598,ds-2,04-STRSAMP-EW,gs://ai-covid19-datalake/trusted/experiment_ma...,48,52868,44538,RF,"{'numTrees': 50, 'maxDepth': 7, 'maxBins': 64}",0.912289736959373,0.9197990889627207,0.889893

In [ ]:
intermed_results.write.parquet('gs://ai-covid19-datalake/trusted/intermed_results/STRSAMP/RF_experiments-ds2.parquet', mode='overwrite')

In [ ]:
print('finished')

finished


In [ ]:
intermed_results.show()

+-----------------+--------------------+--------------------+-------------+-------+-----------+----------+--------------------+------------------+------------------+---------------------+------------------+------------------+-------------------------+----------------------+------------------+-------------------+------------------+------------------+------------------+--------+--------+--------+--------+------------------+-------------+
|experiment_filter|undersampling_method|            filename|experiment_id|n_covid|n_not_covid|model_name|              params|     model_AUC_ROC|      model_AUC_PR|model_covid_precision|model_covid_recall|    model_covid_f1|model_not_covid_precision|model_not_covid_recall|model_not_covid_f1|model_avg_precision|  model_avg_recall|      model_avg_f1|     model_avg_acc|model_TP|model_TN|model_FN|model_FP|   model_time_exec|model_col_set|
+-----------------+--------------------+--------------------+-------------+-------+-----------+----------+--------------